In [16]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import joblib

In [2]:
# Define the path to the Excel file
file_path = r"E:\Data Engineering\Churn_Analysis\Prediction_Data.xlsx"

In [3]:
# Define the sheet name to read data from
sheet_name = 'vw_ChurnData'

In [4]:
# Read the data from the specified sheet into a pandas DataFrame
data = pd.read_excel(file_path, sheet_name=sheet_name)

In [7]:
# Display the first few rows of the fetched data
data.head()

,Customer_ID,Gender,Age,Married,State,Number_of_Referrals,Tenure_in_Months,Value_Deal,Phone_Service,Multiple_Lines,...,Payment_Method,Monthly_Charge,Total_Charges,Total_Refunds,Total_Extra_Data_Charges,Total_Long_Distance_Charges,Total_Revenue,Customer_Status,Churn_Category,Churn_Reason
0,11098-MAD,Female,30,Yes,Madhya Pradesh,0,31,Deal 1,Yes,No,...,Bank Withdrawal,95.099998,6683.399902,0.00,0,631.719971,7315.120117,Stayed,Others,Others
1,11114-PUN,Male,51,No,Punjab,5,9,Deal 5,Yes,No,...,Bank Withdrawal,49.150002,169.050003,0.00,10,122.370003,301.420013,Churned,Competitor,Competitor had better devices
2,11167-WES,Female,43,Yes,West Bengal,3,28,Deal 1,Yes,Yes,...,Bank Withdrawal,116.050003,8297.500000,42.57,110,1872.979980,10237.910156,Stayed,Others,Others
3,11179-MAH,Male,35,No,Maharashtra,10,12,None,Yes,No,...,Credit Card,84.400002,5969.299805,0.00,0,219.389999,6188.689941,Stayed,Others,Others
4,11180-TAM,Male,75,Yes,Tamil Nadu,12,27,Deal 2,Yes,No,...,Credit Card,72.599998,4084.350098,0.00,140,332.079987,4556.430176,Stayed,Others,Others


In [8]:
# Data Preprocessing
# Drop columns that won't be used for prediction
data = data.drop(['Customer_ID', 'Churn_Category', 'Churn_Reason'], axis=1)

In [9]:
# List of columns to be label encoded
columns_to_encode = [
    'Gender', 'Married', 'State', 'Value_Deal', 'Phone_Service', 'Multiple_Lines',
    'Internet_Service', 'Internet_Type', 'Online_Security', 'Online_Backup',
    'Device_Protection_Plan', 'Premium_Support', 'Streaming_TV', 'Streaming_Movies',
    'Streaming_Music', 'Unlimited_Data', 'Contract', 'Paperless_Billing',
    'Payment_Method'
]

In [10]:
# Encode categorical variables except the target variable
label_encoders = {}
for column in columns_to_encode:
    label_encoders[column] = LabelEncoder()
    data[column] = label_encoders[column].fit_transform(data[column])

In [11]:
# Manually encode the target variable 'Customer_Status'
data['Customer_Status'] = data['Customer_Status'].map({'Stayed': 0, 'Churned': 1})

In [12]:
# Split data into features and target
X = data.drop('Customer_Status', axis=1)
y = data['Customer_Status']

In [13]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [18]:
# Train Logistic Regression Model with increased iterations and scaled data
lr_model = LogisticRegression(max_iter=5000, solver='saga', random_state=42)
lr_model.fit(X_train_scaled, y_train)


LogisticRegression(max_iter=5000, random_state=42, solver='saga')

In [19]:
# Evaluate Model
# Make predictions
y_pred_lr = lr_model.predict(X_test_scaled)

In [20]:
# Evaluate Logistic Regression Model
print("Logistic Regression Model:")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_lr))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_lr))

Logistic Regression Model:
Confusion Matrix:
[[726 121]
 [127 228]]

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.86      0.85       847
           1       0.65      0.64      0.65       355

    accuracy                           0.79      1202
   macro avg       0.75      0.75      0.75      1202
weighted avg       0.79      0.79      0.79      1202



In [49]:
# Train Random Forest Model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)


RandomForestClassifier(random_state=42)

In [50]:
# Evaluate Model
# Make predictions
y_pred_rf = rf_model.predict(X_test_scaled)

In [51]:
# Evaluate Random Forest Model
print("Random Forest Model:")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf))


Random Forest Model:
Confusion Matrix:
[[783  64]
 [125 230]]

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.92      0.89       847
           1       0.78      0.65      0.71       355

    accuracy                           0.84      1202
   macro avg       0.82      0.79      0.80      1202
weighted avg       0.84      0.84      0.84      1202



In [24]:
# Hyperparameter tuning for Logistic Regression using GridSearchCV
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [1000, 5000, 10000]
}

In [25]:
grid_lr = GridSearchCV(estimator=lr_model, param_grid=param_grid_lr, cv=3, n_jobs=-1, verbose=2)
grid_lr.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 60 candidates, totalling 180 fits


GridSearchCV(cv=3,
             estimator=LogisticRegression(max_iter=5000, random_state=42,
                                          solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10, 100],
                         'max_iter': [1000, 5000, 10000],
                         'solver': ['lbfgs', 'liblinear', 'sag', 'saga']},
             verbose=2)

In [26]:
print("Best parameters for Logistic Regression:", grid_lr.best_params_)
y_pred_lr_tuned = grid_lr.predict(X_test_scaled)

Best parameters for Logistic Regression: {'C': 100, 'max_iter': 1000, 'solver': 'sag'}


In [27]:
print("Tuned Logistic Regression Model:")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_lr_tuned))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_lr_tuned))

Tuned Logistic Regression Model:
Confusion Matrix:
[[726 121]
 [127 228]]

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.86      0.85       847
           1       0.65      0.64      0.65       355

    accuracy                           0.79      1202
   macro avg       0.75      0.75      0.75      1202
weighted avg       0.79      0.79      0.79      1202



In [52]:
# Hyperparameter tuning for Random Forest using GridSearchCV
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_features': ['sqrt', 'log2'],  # Updated to remove 'auto'
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [53]:
grid_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid_rf, cv=3, n_jobs=-1, verbose=2)
grid_rf.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [10, 20, 30, None],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             verbose=2)

In [54]:
print("Best parameters for Random Forest:", grid_rf.best_params_)
y_pred_rf_tuned = grid_rf.predict(X_test_scaled)

Best parameters for Random Forest: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [55]:
print("Tuned Random Forest Model:")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf_tuned))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf_tuned))

Tuned Random Forest Model:
Confusion Matrix:
[[785  62]
 [126 229]]

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.93      0.89       847
           1       0.79      0.65      0.71       355

    accuracy                           0.84      1202
   macro avg       0.82      0.79      0.80      1202
weighted avg       0.84      0.84      0.84      1202



In [56]:
# Compare Models
print("Model Comparison:")
print("Random Forest Accuracy:", grid_rf.best_score_)
print("Logistic Regression Accuracy:", grid_lr.best_score_)

Model Comparison:
Random Forest Accuracy: 0.8574396776047429
Logistic Regression Accuracy: 0.823099274979771


In [36]:
# Use Model for Prediction on New Data

In [37]:
# Define the path to the Joiner Data Excel file
file_path = r"E:\Data Engineering\Churn_Analysis\Prediction_data.xlsx"

In [38]:
# Define the sheet name to read data from
sheet_name = 'vw_JoinData'

In [39]:
# Read the data from the specified sheet into a pandas DataFrame
new_data = pd.read_excel(file_path, sheet_name=sheet_name)

In [40]:
# Display the first few rows of the fetched data
new_data.head()


,Customer_ID,Gender,Age,Married,State,Number_of_Referrals,Tenure_in_Months,Value_Deal,Phone_Service,Multiple_Lines,...,Payment_Method,Monthly_Charge,Total_Charges,Total_Refunds,Total_Extra_Data_Charges,Total_Long_Distance_Charges,Total_Revenue,Customer_Status,Churn_Category,Churn_Reason
0,11751-TAM,Female,18,No,Tamil Nadu,5,7,Deal 5,No,No,...,Mailed Check,24.299999,38.450001,0.0,0,0.000000,38.450001,Joined,Others,Others
1,12056-WES,Male,27,No,West Bengal,2,20,None,Yes,No,...,Bank Withdrawal,90.400002,268.450012,0.0,0,94.440002,362.890015,Joined,Others,Others
2,12136-RAJ,Female,25,Yes,Rajasthan,2,35,None,Yes,No,...,Bank Withdrawal,19.900000,19.900000,0.0,0,11.830000,31.730000,Joined,Others,Others
3,12257-ASS,Female,39,No,Assam,9,1,None,Yes,No,...,Credit Card,19.549999,19.549999,0.0,0,10.200000,29.750000,Joined,Others,Others
4,12340-DEL,Female,51,Yes,Delhi,0,10,None,Yes,No,...,Credit Card,62.799999,62.799999,0.0,0,42.189999,104.989998,Joined,Others,Others


In [41]:
# Retain the original DataFrame to preserve unencoded columns
original_data = new_data.copy()

In [42]:
# Retain the Customer_ID column
customer_ids = new_data['Customer_ID']

In [43]:
# Drop columns that won't be used for prediction in the encoded DataFrame
new_data = new_data.drop(['Customer_ID', 'Customer_Status', 'Churn_Category', 'Churn_Reason'], axis=1)

In [44]:
# Encode categorical variables using the saved label encoders
for column in new_data.select_dtypes(include=['object']).columns:
    new_data[column] = label_encoders[column].transform(new_data[column])

In [45]:
# Make predictions
new_predictions = grid_rf.predict(new_data)

In [46]:
# Add predictions to the original DataFrame
original_data['Customer_Status_Predicted'] = new_predictions

In [47]:
# Filter the DataFrame to include only records predicted as "Churned"
original_data = original_data[original_data['Customer_Status_Predicted'] == 1]

In [48]:
# Save the results
original_data.to_csv(r"E:\Data Engineering\Churn_Analysis\Predictions_output.csv", index=False)